This is a base jupyter notebook for other solutions. It goes through the dataset A, and it finds the highest and the lowest energy production. It generates the solution with random floats inbetween the highest and the lowest production.

In [140]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [141]:
%autoreload

# load libraries
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR


In [142]:
# read datasets
# for simplicity, I use X_train_estimated as test data for cross validation
y = pd.read_parquet("../../dataset/A/train_targets.parquet")
X_train = pd.read_parquet("../../dataset/A/X_train_observed.parquet")
X_test = pd.read_parquet("../../dataset/A/X_train_estimated.parquet")

# y is energy production every hour, but X is weather every 15 minutes
X_train = X_train[X_train.index % 4 == 0]
X_train.reset_index(drop=True, inplace=True)
X_test = X_test[X_test.index % 4 == 0]
X_test.reset_index(drop=True, inplace=True)

# split y to train and test set
n_train_rows = X_train.shape[0]
n_test_rows = X_test.shape[0]
y_train = y.iloc[:n_train_rows]
y_test = y.iloc[-n_test_rows:]

In [143]:
print(f"X_train.shape = {X_train.shape}")
print(f"X_test.shape = {X_test.shape}")
print(f"y_train.shape = {y_train.shape}")
print(f"y_test.shape = {y_test.shape}")

X_train.shape = (29668, 46)
X_test.shape = (4394, 47)
y_train.shape = (29668, 2)
y_test.shape = (4394, 2)


In [144]:
# edit data

# remove date_calc column, because it is useless for energy production forecasting
X_test.drop("date_calc", axis=1, inplace=True)

# remove date from y_train and y_test
y_train.drop("time", axis=1, inplace=True)
y_test.drop("time", axis=1, inplace=True)

# split "date_forecast" feature into "day of the year" and "time of the day" in X_train
X_train['day_of_year'] = X_train['date_forecast'].apply(lambda x: x.dayofyear)
X_train['hour_of_day'] = X_train['date_forecast'].apply(lambda x: x.hour)
X_train.drop('date_forecast', axis=1, inplace=True)

# split "date_forecast" feature into "day of the year" and "time of the day" in X_test
X_test['day_of_year'] = X_test['date_forecast'].apply(lambda x: x.dayofyear)
X_test['hour_of_day'] = X_test['date_forecast'].apply(lambda x: x.hour)
X_test.drop('date_forecast', axis=1, inplace=True)

# there are NaNs in X_train and X_test (feature "snow_density"), replace them for zeros
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

/tmp/ipykernel_36604/4018083949.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_train.drop("time", axis=1, inplace=True)
/tmp/ipykernel_36604/4018083949.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_test.drop("time", axis=1, inplace=True)


## Machine learning stuff

In [145]:
# decision tree
decision_tree = DecisionTreeRegressor()
decision_tree.fit(X_train, y_train)
y_pred_tree = decision_tree.predict(X_test)

mae_tree = np.mean(np.abs(np.array(y_test) - y_pred_tree))
mae_tree

617.9890257921948

In [146]:
# random forest
random_forest = RandomForestRegressor(n_estimators=100)  # You can adjust the number of trees (n_estimators) as needed
random_forest.fit(X_train, y_train)
y_pred_forest = random_forest.predict(X_test)

mae_forest = np.mean(np.abs(np.array(y_test) - y_pred_forest))
mae_forest

/tmp/ipykernel_36604/3459638483.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  random_forest.fit(X_train, y_train)


599.8973885900494

In [147]:
# gradient boosting
gradient_boosting = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1)  # You can adjust the number of trees (n_estimators) as needed
gradient_boosting.fit(X_train, y_train)
y_pred_grad = gradient_boosting.predict(X_test)

mae_grad = np.mean(np.abs(np.array(y_test) - y_pred_grad))
mae_grad

/home/karel/.local/lib/python3.10/site-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


591.932941522667

In [148]:
# elastic net
elastic_net = ElasticNet(alpha=1.0, l1_ratio=0.5, random_state=42)
elastic_net.fit(X_train, y_train)
y_pred_elast_net = elastic_net.predict(X_test)

mae_elast_net = np.mean(np.abs(np.array(y_test) - y_pred_elast_net))
mae_elast_net

/home/karel/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.684e+09, tolerance: 4.240e+06
  model = cd_fast.enet_coordinate_descent(


599.6075209008663

In [149]:
# support vector regression
svr_model = SVR(kernel='rbf', C=1.0)
svr_model.fit(X_train, y_train)
y_pred_svr = svr_model.predict(X_test)

mae_svr = np.mean(np.abs(np.array(y_test) - y_pred_svr))
mae_svr

/home/karel/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


473.3314940446853

SVR model came out with the lowest mean absolute error. So far we did only cross validation on the training data. SVR will be used on the real test data, on the datasets B and C and to generate the output csv file.

In [152]:
# first dataset
X_test = pd.read_parquet("../../dataset/A/X_test_estimated.parquet")
X_test.head()
# TODO: sorry, not finished
# TODO: join data X_train_observed, X_train_estimated and train SVR on that, predict on X_test_estimated.
# TODO: do the same thing on the datasets B and C
# TODO: put it the results to results.csv and upload on kaggle

,date_calc,date_forecast,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,...,sun_azimuth:d,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms
0,2023-04-30 07:00:04,2023-05-01 00:00:00,4.4,1.286,912.700012,0.0,0.0,1041.199951,0.0,271.700012,...,10.520000,-11.185,0.0,273.799988,80.699997,30210.699219,4.0,2.2,3.4,-0.0
1,2023-04-30 07:00:04,2023-05-01 00:15:00,4.3,1.287,912.700012,0.0,0.0,1054.800049,0.0,271.700012,...,14.203000,-10.825,0.0,273.799988,77.000000,30003.599609,4.0,2.1,3.4,-0.0
2,2023-04-30 07:00:04,2023-05-01 00:30:00,4.3,1.287,912.700012,0.0,0.0,1068.300049,0.0,271.600006,...,17.868999,-10.360,0.0,273.799988,73.099998,29797.099609,3.9,2.1,3.3,-0.0
3,2023-04-30 07:00:04,2023-05-01 00:45:00,4.3,1.287,912.700012,0.0,0.0,1081.900024,0.0,271.600006,...,21.514000,-9.794,0.0,273.799988,69.000000,29618.599609,3.9,2.0,3.3,-0.0
4,2023-04-30 07:00:04,2023-05-01 01:00:00,4.3,1.287,NaN,0.0,0.0,1095.400024,0.0,271.600006,...,25.135000,-9.128,0.0,273.799988,64.500000,29507.500000,3.9,2.0,3.3,-0.0
